todo-list
=========
<li>Migrate static->stream(if possible)</li>
<li>Find new efficient way to do pattern detection/anomaly detection</li>
<li>Email alert config (now sent to yanisa.sunthornyotin@cern.ch)</li>

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
userSchema = StructType() \
        .add("window",StructType()\
             .add("start",TimestampType())\
             .add("end",TimestampType()))\
        .add("system", StringType())\
        .add("api", StringType())\
        .add("user", StringType())\
        .add("count_req", LongType())

In [2]:
raw_data = spark\
.readStream.format("parquet")\
.schema(userSchema)\
.load("/cms/users/carizapo/ming/data_cmsweb_logs");
raw_data.printSchema()

root
 |-- window: struct (nullable = true)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- system: string (nullable = true)
 |-- api: string (nullable = true)
 |-- user: string (nullable = true)
 |-- count_req: long (nullable = true)



In [3]:
temp_data = spark\
.read.format("parquet")\
.load("/cms/users/carizapo/ming/data_cmsweb_logs");
temp_data.printSchema()

root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- system: string (nullable = true)
 |-- api: string (nullable = true)
 |-- user: string (nullable = true)
 |-- count_req: long (nullable = false)



In [4]:
temp_data.show()

+--------------------+-------------+--------------------+--------------------+---------+
|              window|       system|                 api|                user|count_req|
+--------------------+-------------+--------------------+--------------------+---------+
|[2019-08-05 14:29...|   crabserver|   fileusertransfers|/DC=ch/DC=cern/OU...|      172|
|[2019-08-05 14:29...|      couchdb|cmsunified_ACDC2_...|/DC=org/DC=incomm...|        1|
|[2019-08-05 14:28...|wmstatsserver|pdmvserv_task_EXO...|/DC=ch/DC=cern/OU...|        1|
|[2019-08-05 14:28...|      couchdb|pdmvserv_task_B2G...|/DC=ch/DC=cern/OU...|        1|
|[2019-08-05 14:28...|      couchdb|pdmvserv_task_B2G...|/DC=ch/DC=cern/OU...|        1|
|[2019-08-05 14:29...|      couchdb|pdmvserv_task_B2G...|/DC=ch/DC=cern/OU...|        3|
|[2019-08-05 14:28...|      couchdb|pdmvserv_task_EXO...|/DC=ch/DC=cern/OU...|        1|
|[2019-08-05 14:29...|      reqmgr2|pdmvserv_task_HIG...|/DC=ch/DC=cern/OU...|        1|
|[2019-08-05 14:29...

Calculate % Difference according to the mean of the amounth of email within 1 day
-----------

In [13]:
from pyspark.sql import Window
obs_interval=window("window.start", "1 hour")
mean_interval=window("window.start", "1 day")

columns_drop=['diff_user','diff_api','diff_sys','diff_req']

wo_sys = Window.partitionBy('system',obs_interval)
wo_user = Window.partitionBy('user',obs_interval)
wo_api = Window.partitionBy('api',obs_interval)
wo_req = Window.partitionBy('system','user','api',obs_interval)


wm_sys = Window.partitionBy('system',mean_interval)
wm_user = Window.partitionBy('user',mean_interval)
wm_api = Window.partitionBy('api',mean_interval)
wm_req = Window.partitionBy('system','user','api',mean_interval)

freq_analyze_df=temp_data\
.filter("user!='null' and user!='-'")\
.select('*', sum('count_req').over(wo_req).alias('req_load'))\
.select('*', count('system').over(wo_sys).alias('system_load'))\
.select('*', count('api').over(wo_api).alias('api_load'))\
.select('*', count('user').over(wo_user).alias('user_load'))
freq_analyze_df=freq_analyze_df\
.select('*', avg('req_load').over(wm_req).alias('avg_req'))\
.select('*', ((col('req_load') - first('avg_req').over(wo_req))).alias('diff_req'))\
.select('*', ((col('diff_req')/first('avg_req').over(wo_req))).alias('%diff_req'))\
.select('*', avg('system_load').over(wm_sys).alias('avg_sys'))\
.select('*', ((col('system_load') - first('avg_sys').over(wo_sys))).alias('diff_sys'))\
.select('*', ((col('diff_sys')/first('avg_sys').over(wo_sys))).alias('%diff_sys'))\
.select('*', avg('api_load').over(wm_api).alias('avg_api'))\
.select('*', ((col('api_load') - first('avg_api').over(wo_api))).alias('diff_api'))\
.select('*', ((col('diff_api')/first('avg_api').over(wo_api))).alias('%diff_api'))\
.select('*', avg('user_load').over(wm_user).alias('avg_user'))\
.select('*', ((col('user_load') - first('avg_user').over(wo_user))).alias('diff_user'))\
.select('*', ((col('diff_user')/first('avg_user').over(wo_user))).alias('%diff_user'))\
.drop(*columns_drop)
#Todos:Transform/Refactor to UDF

In [14]:
joined_raw_data=raw_data.join(freq_analyze_df, ["system","window","api","user","count_req"], "inner")

In [ ]:
joined_raw_data.printSchema()

In [17]:
full_difference_hdfs=joined_raw_data.writeStream\
.outputMode("append")\
.format("parquet")\
 .option("path", "/cms/users/carizapo/ming/fullDiff_cmsweb_logs") \
 .option("checkpointLocation", "/cms/users/carizapo/ming/checkpoint_prep_cmsweb_logs") \
 .option("failOnDataLoss",False)\
 .outputMode("append") \
 .start()

In [18]:
spark.streams.active

In [19]:
full_difference_hdfs.processAllAvailable()

StreamingQueryException: 'Job aborted.\n=== Streaming Query ===\nIdentifier: [id = 1c8560e2-d013-4c99-96d5-12518f383679, runId = 53cd779e-0b39-4201-abb2-fbc1c50cc13d]\nCurrent Committed Offsets: {FileStreamSource[hdfs://analytix/cms/users/carizapo/ming/data_cmsweb_logs]: {"logOffset":8}}\nCurrent Available Offsets: {FileStreamSource[hdfs://analytix/cms/users/carizapo/ming/data_cmsweb_logs]: {"logOffset":9}}\n\nCurrent State: ACTIVE\nThread State: RUNNABLE\n\nLogical Plan:\nProject [system#392, window#391, api#393, user#394, count_req#395L, req_load#414L, system_load#425L, api_load#437L, user_load#450L, avg_req#463, %diff_req#493, avg_sys#510, %diff_sys#546, avg_api#566, %diff_api#608, avg_user#631, %diff_user#679]\n+- Join Inner, (((((system#392 = system#402) && (window#391 = window#401-T60000ms)) && (api#393 = api#403)) && (user#394 = user#404)) && (count_req#395L = count_req#405L))\n   :- StreamingExecutionRelation FileStreamSource[hdfs://analytix/cms/users/carizapo/ming/data_cmsweb_

In [ ]:
full_difference_hdfs.stop()